<a href="https://colab.research.google.com/github/GalileyGalileo/TelegramBotTest/blob/main/TelegramBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import telebot
from telebot import types
import os
import datetime

# Ваш токен бота Telegram
TOKEN = 'YOUR_BOT_TOKEN'
bot = telebot.TeleBot(TOKEN)

# Словарь для хранения счетчика книг, даты и статистики для каждого пользователя в каждой группе
user_data = {}

# Функция для отправки ежедневного напоминания
def send_daily_reminder(chat_id, user_id):
    if chat_id not in user_data:
        user_data[chat_id] = {}

    if user_id not in user_data[chat_id]:
        user_data[chat_id][user_id] = {'book_count': 2, 'last_date': '', 'stats': {'duolingo': 0, 'read_chapter': 0}}

    book_count = user_data[chat_id][user_id]['book_count']
    last_date = user_data[chat_id][user_id]['last_date']

    today = datetime.date.today()
    today_str = today.strftime('%d.%m.%Y')

    if last_date != today_str:
        # Если сегодняшняя дата отличается от последней сохраненной даты, обновляем дату и счетчик
        user_data[chat_id][user_id]['last_date'] = today_str
        user_data[chat_id][user_id]['book_count'] += 1

    book_count = user_data[chat_id][user_id]['book_count']

    reminder_text = f"Тебе нужно прочитать {book_count} главы книги и сделать Duolingo сегодня, {today_str}."

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    button_read_book = types.KeyboardButton("Прочитал главу книги")
    button_duolingo = types.KeyboardButton("Сделал Duolingo")
    markup.add(button_read_book, button_duolingo)
    bot.send_message(chat_id, reminder_text, reply_markup=markup)

# Обработчик команды "Старт" для начала взаимодействия с ботом
@bot.message_handler(commands=['start'])
def handle_start(message):
    chat_id = message.chat.id
    user_id = message.from_user.id
    send_daily_reminder(chat_id, user_id)

# Обработчик для кнопки "Прочитал главу книги"
@bot.message_handler(func=lambda message: message.text == "Прочитал главу книги")
def handle_read_book(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id not in user_data:
        user_data[chat_id] = {}

    if user_id not in user_data[chat_id]:
        user_data[chat_id][user_id] = {'book_count': 2, 'last_date': '', 'stats': {'duolingo': 0, 'read_chapter': 0}}

    user_data[chat_id][user_id]['book_count'] -= 1

    # Увеличиваем счетчик "Прочитал главу" в статистике
    user_data[chat_id][user_id]['stats']['read_chapter'] += 1

    send_daily_reminder(chat_id, user_id)

# Обработчик для кнопки "Сделал Duolingo"
@bot.message_handler(func=lambda message: message.text == "Сделал Duolingo")
def handle_duolingo(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id not in user_data:
        user_data[chat_id] = {}

    if user_id not in user_data[chat_id]:
        user_data[chat_id][user_id] = {'book_count': 2, 'last_date': '', 'stats': {'duolingo': 0, 'read_chapter': 0}}

    user_data[chat_id][user_id]['book_count'] += 1
    user_data[chat_id][user_id]['last_date'] = ''

    response = bot.send_message(chat_id, "Пришли скриншот выполненного Duolingo, пожалуйста.")

    bot.register_next_step_handler(response, process_duolingo_screenshot)

# Обработчик скриншота Duolingo
def process_duolingo_screenshot(message):
    chat_id = message.chat.id
    user_id = message.from_user.id
    if message.photo:
        # Если прислан фото, обрабатываем скриншот
        user_data[chat_id][user_id]['last_date'] = ''  # Сбрасываем дату
        bot.send_message(chat_id, "Ура, молодец, счет записан.")

        # Увеличиваем счетчик "Duolingo" в статистике
        user_data[chat_id][user_id]['stats']['duolingo'] += 1
    else:
        # Если прислано что-то другое, просим прислать скриншот еще раз
        bot.send_message(chat_id, "Неправильно! Пришли скриншот выполненного Duolingo, пожалуйста.")

# Обработчик команды "/stat"
@bot.message_handler(commands=['stat'])
def handle_stats(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id in user_data and user_id in user_data[chat_id]:
        stats = user_data[chat_id][user_id]['stats']
        stats_text = f"Твоя статистика:\nDuolingo: {stats['duolingo']} раз\nПрочитана глава: {stats['read_chapter']} раз"
        bot.send_message(chat_id, stats_text)
    else:
        bot.send_message(chat_id, "У тебя пока нет статистики.")

# Обработчик команды "/add"
@bot.message_handler(commands=['add'])
def handle_add(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id in user_data and user_id in user_data[chat_id]:
        user_data[chat_id][user_id]['book_count'] += 1
        bot.send_message(chat_id, "День успешно добавлен к счетчику.")
        send_daily_reminder(chat_id, user_id)
    else:
        bot.send_message(chat_id, "У тебя пока нет статистики.")

# Обработчик команды "/delete"
@bot.message_handler(commands=['delete'])
def handle_delete(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id in user_data and user_id in user_data[chat_id]:
        if user_data[chat_id][user_id]['book_count'] > 1:
            user_data[chat_id][user_id]['book_count'] -= 1
            bot.send_message(chat_id, "День успешно удален из счетчика.")
            send_daily_reminder(chat_id, user_id)
        else:
            bot.send_message(chat_id, "Счетчик не может быть меньше 1.")
    else:
        bot.send_message(chat_id, "У тебя пока нет статистики.")

# Обработчик команды "/deletestats"
@bot.message_handler(commands=['deletestats'])
def handle_delete_stats(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if chat_id in user_data and user_id in user_data[chat_id]:
        response = bot.send_message(chat_id, "Вы уверены, что хотите сбросить свою статистику? Это действие нельзя отменить.",
                                   reply_markup=types.ReplyKeyboardRemove())
        bot.register_next_step_handler(response, confirm_delete_stats)
    else:
        bot.send_message(chat_id, "У тебя пока нет статистики.")

# Функция для подтверждения сброса статистики
def confirm_delete_stats(message):
    chat_id = message.chat.id
    user_id = message.from_user.id

    if message.text.lower() == 'да':
        if chat_id in user_data and user_id in user_data[chat_id]:
            user_data[chat_id][user_id] = {'book_count': 2, 'last_date': '', 'stats': {'duolingo': 0, 'read_chapter': 0}}
            bot.send_message(chat_id, "Статистика успешно сброшена.")
        else:
            bot.send_message(chat_id, "У тебя пока нет статистики.")
    else:
        bot.send_message(chat_id, "Статистика не была сброшена.")

if __name__ == "__main__":
    bot.polling()
